---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    # extract date format 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    extracted = df.str.extractall(r'(?P<Full_date>(?P<month>\d?\d)[/-](?P<day>\d?\d)[/-](?P<year>\d{2,4}))')
    
    # get a true list index for index is not in extracted dataframe
   # there are two index (the second index is 'match'), x[0] is the first index, which we want
    index_left = ~df.index.isin([x[0] for x in extracted.index])
    
    # from unextracted index dataframe, extract Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    extracted_1 = df[index_left].str.extractall(r'(?P<Full_date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ -.](?P<day>\d{1,2})[, ](?P<year>\d{2,4}))')
   # Change the columns order to match the extracted dataframe
    extracted_1 = extracted_1[['Full_date','day','month','year']]
   # append the extracted_1 to extracted dataframe
    extracted = extracted.append(extracted_1)
    index_left = ~df.index.isin([x[0] for x in extracted.index])
    
    # from unextracted index dataframe, extract 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    extracted_2 = df[index_left].str.extractall(r'(?P<Full_date>(?P<day>\d{1,2}) (?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[. ,](?P<year>\d{2,4}))')
   # Change the columns order to match the extracted dataframe
    extracted_2 = extracted_2[['Full_date','day','month','year']]
   # append the extracted_2 to extracted dataframe
    extracted = extracted.append(extracted_2)
    index_left = ~df.index.isin([x[0] for x in extracted.index])
    
    # extract date format Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009 (no result was found)
    extracted_3 = df[index_left].str.extractall(r'(?P<Full_date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?P<day>\d{1,2})[a-z]*,(?P<year>\d{2,4}))')
   # Change the columns order to match the extracted dataframe
    extracted_3 = extracted_3[['Full_date','day','month','year']]
   # append the extracted_3 to extracted dataframe
    extracted = extracted.append(extracted_3)
    index_left = ~df.index.isin([x[0] for x in extracted.index])
    
    # from unextracted index dataframe, extract Feb 2009; Sep 2009; Oct 2010
    extracted_4 = df[index_left].str.extractall(r'(?P<Full_date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ,.](?P<year>\d{2,4}))')
    extracted_4['day'] = 1
   # Change the columns order to match the extracted dataframe
    extracted_4 = extracted_4[['Full_date','day','month','year']]
   # append the extracted_4 to extracted dataframe
    extracted = extracted.append(extracted_4)
    index_left = ~df.index.isin([x[0] for x in extracted.index])
    
    # from unextracted index dataframe, extract 6/2008; 12/2009
    extracted_5 = df[index_left].str.extractall(r'(?P<Full_date>(?P<month>\d{1,2})/(?P<year>\d{4}))')
    extracted_5['day'] = 1
    extracted_5 = extracted_5[['Full_date','day','month','year']]
    extracted = extracted.append(extracted_5)
    index_left = ~df.index.isin([x[0] for x in extracted.index])
    
    # from unextracted index dataframe, extract 2008; 2009
    extracted_6 = df[index_left].str.extractall(r'(?P<Full_date>(?P<year>\d{4}))')
    extracted_6['day'] = 1
    extracted_6['month'] = 1
    extracted_6 = extracted_6[['Full_date','day','month','year']]
    extracted = extracted.append(extracted_6)
    
    # change two number year to four number year
    extracted['year'] = extracted['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
    
    # remoce 0 in front of number, for example change 07 to 7
    extracted['month'] = extracted['month'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x)
    # replace word month with number
    month_dict = dict({'Sep': 9, 'Mar': 3, 'Nov': 11, 'Jul': 7, 'Jan': 1, 'Dec': 12,
                       'Feb': 2, 'May': 5, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Jun': 6, 'Apr': 4})
    extracted.replace({'month': month_dict}, inplace=True)
    
    # make sure all 'month' and 'day' are str type
    extracted['month'] = extracted['month'].apply(lambda x: str(x))
    extracted['day'] = extracted['day'].apply(lambda x: str(x))
    
    # connect month, day, and year together by /
    extracted['date'] = extracted['month'] + '/' + extracted['day'] + '/' + extracted['year']
    # need to add errors='coerce', or some days will out of range in month, and get error, e.g 03/40/2018
    extracted['date'] = pd.to_datetime(extracted['date'], errors='coerce')
    
    # sort the dates
    extracted.sort_values(by='date', inplace=True)
    
    # there are two index, the first one 'label[0]' is what we want. 
    date_order = pd.Series(list(extracted.index.labels[0]))
    
    return date_order
    
date_sorter()

0      208
1      203
2      226
3      194
4      213
5      210
6      216
7      217
8      218
9      215
10     202
11     222
12     225
13     211
14     197
15     196
16     221
17     201
18     212
19     220
20     209
21     224
22     200
23     205
24     195
25       9
26      84
27       2
28      53
29      28
      ... 
472    243
473    139
474    320
475    383
476    286
477    244
478    198
479    480
480    431
481    279
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
500     72
501    204
Length: 502, dtype: int64